<p style="float:right">
<img src="images/logos/cu.png" style="display:inline" />
<img src="images/logos/cires.png" style="display:inline" />
<img src="images/logos/nasa.png" style="display:inline" />
<img src="images/logos/nsidc_daac.png" style="display:inline" />
</p>

# Python, Jupyter & pandas: Module 3

## Viewing geolocated data

In Module 2, we retrieved NSIDC snow cover data in the northern hemisphere from an OPeNDAP server. Let's pull that data again for use in this notebook:

In [ ]:
import netCDF4
import numpy as np

url_basic = ('http://opendap.apps.nsidc.org:80/opendap/DATASETS/'
             'nsidc0530_MEASURES_nhsnow_daily25/2012/nhtsd25e2_20120101_v01r01.nc?'
             'latitude[0:1:719][0:1:719],'
             'longitude[0:1:719][0:1:719],'
             'merged_snow_cover_extent[0:1:0][0:1:719][0:1:719]'
)

dataset_basic = netCDF4.Dataset(url_basic)

latitude_basic = np.array(dataset_basic.variables['latitude'])
longitude_basic = np.array(dataset_basic.variables['longitude'])
msce_basic = np.array(dataset_basic.variables['merged_snow_cover_extent'])[0, :, :]

And now let's pull in some graphics support now and get ready to plot our snow cover data.

In [ ]:
%matplotlib inline
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

It will be helpful to have an underlying map image to better interpret our data, so let's be sure we can at least do that with `matplotlib`'s [`Basemap`](http://matplotlib.org/basemap/api/basemap_api.html?highlight=basemap#module-mpl_toolkits.basemap):

In [ ]:
plt.figure(figsize=(10, 10))
m = Basemap(projection='npstere', boundinglat=45, lon_0=300)
m.drawcoastlines()
plt.draw()

We can ask for more interesting background imagery (we won't, in general, in this tutorial, in order to focus on the data instead of the map):

In [ ]:
plt.figure(figsize=(10, 10))
m = Basemap(projection='npstere', boundinglat=45, lon_0=300)
m.drawcoastlines()
m.etopo()
plt.draw()

Let's take a quick look back at the metadata for our Merged Snow Cover Extent variable:

In [ ]:
dataset_basic.variables['merged_snow_cover_extent']

So, the snow cover variable is categorical (discrete), and consists of integers whose values represent snow cover from different sources, snow-free land, permanent ice-covered land, ocean, or a fill value.

As a reality check, let's find the would-be ocean points and plot them on our map:

In [ ]:
plt.figure(figsize=(10, 10))
m = Basemap(projection='npstere', boundinglat=45, lon_0=300)
m.drawcoastlines()

row, col = np.where(msce_basic == 40)
x, y = m(longitude_basic[row, col], latitude_basic[row, col])
m.plot(x, y, '.', color='SeaGreen')

plt.draw()

Seems legit.

Now, where's the snow? Values 10 through 16 in the merged snow cover variable all indicate snow, so we'll accept anything in that range. In fact, since there are no non-snow values less than 17, we can just use that as our condition:

In [ ]:
plt.figure(figsize=(10, 10))
m = Basemap(projection='npstere', boundinglat=45, lon_0=300)
m.drawcoastlines()

row, col = np.where(msce_basic <= 17)
x, y = m(longitude_basic[row, col], latitude_basic[row, col])
m.plot(x, y, '.', color='Cyan')

plt.draw()

How about snow **or** permanent ice covered land?

In [ ]:
plt.figure(figsize=(10, 10))
m = Basemap(projection='npstere', boundinglat=45, lon_0=300)
m.drawcoastlines()

snow = (msce_basic <= 17)
ice = (msce_basic == 30)

snow_col, snow_row = np.where(snow)
ice_col, ice_row = np.where(ice)

snow_x, snow_y = m(longitude_basic[snow_col, snow_row], latitude_basic[snow_col, snow_row])
m.plot(snow_x, snow_y, '.', color='Cyan')

ice_x, ice_y = m(longitude_basic[ice_col, ice_row], latitude_basic[ice_col, ice_row])
m.plot(ice_x, ice_y, '.', color='Blue')

plt.draw()

At the end of Module 2, we tried to use OPeNDAP to retrieve data only in the area of Iceland. Let's execute that query again:

In [ ]:
iceland_url = ('http://opendap.apps.nsidc.org:80/opendap/DATASETS/'
               'nsidc0530_MEASURES_nhsnow_daily25/2012/nhtsd25e2_20120101_v01r01.nc?'
               'latitude[453:1:476][310:1:338],'
               'longitude[453:1:476][310:1:338],'
               'merged_snow_cover_extent[0:1:0][453:1:476][310:1:338]'
)

iceland_dataset = netCDF4.Dataset(iceland_url)

iceland_latitude = np.array(iceland_dataset.variables['latitude'])
iceland_longitude = np.array(iceland_dataset.variables['longitude'])
iceland_msce = np.array(iceland_dataset.variables['merged_snow_cover_extent'])[0, :, :]

And now let's plot it on a map and see if the result makes sense:

In [ ]:
plt.figure(figsize=(10, 10))
m = Basemap(projection='npstere', boundinglat=45, lon_0=300)
m.drawcoastlines()

snow = (iceland_msce <= 17)
ice = (iceland_msce == 30)

snow_col, snow_row = np.where(snow)
ice_col, ice_row = np.where(ice)

snow_x, snow_y = m(iceland_longitude[snow_col, snow_row], iceland_latitude[snow_col, snow_row])
m.plot(snow_x, snow_y, '.', color='Cyan')

ice_x, ice_y = m(iceland_longitude[ice_col, ice_row], iceland_latitude[ice_col, ice_row])
m.plot(ice_x, ice_y, '.', color='Blue')

plt.draw()

Looks about right, but let's get a closer look. Can we plot a map of just Iceland?

In [ ]:
plt.figure(figsize=(10, 10))
m = Basemap(projection='merc', llcrnrlon=-25, llcrnrlat=63, urcrnrlon=-13, urcrnrlat=67, resolution='i')
m.drawcoastlines()
plt.draw()

Let's plot the snow- and ice-cover points. We can re-use the `[snow|ice]_[col|row]` variables we computed previously, but need to recompute the coordinates since we are using a different map projection (Mercator, instead of North Polar Stereographic).

In [ ]:
plt.figure(figsize=(10, 10))
m = Basemap(projection='merc', llcrnrlon=-25, llcrnrlat=63, urcrnrlon=-13, urcrnrlat=67, resolution='i')
m.drawcoastlines()

snow_x, snow_y = m(iceland_longitude[snow_col, snow_row], iceland_latitude[snow_col, snow_row])
m.plot(snow_x, snow_y, '.', color='Cyan', markersize='24')

ice_x, ice_y = m(iceland_longitude[ice_col, ice_row], iceland_latitude[ice_col, ice_row])
m.plot(ice_x, ice_y, '.', color='Blue', markersize='24')

plt.draw()

If we add some (admittedly lo-res) background imagery, we can see that the permanent ice is located in what appear to be sensible places:

In [ ]:
plt.figure(figsize=(10, 10))
m = Basemap(projection='merc', llcrnrlon=-25, llcrnrlat=63, urcrnrlon=-13, urcrnrlat=67, resolution='i')
m.plot(snow_x, snow_y, '.', color='Cyan', markersize='24')
m.plot(ice_x, ice_y, '.', color='Blue', markersize='24')
m.bluemarble()
plt.draw()

There are certainly more sophisticated ways to visualize these data. In the `module-3-extra` notebook in the `extra` folder, there's an example of something a bit nicer. But hopefully this gives an idea of what's possible.

In Module 4, we'll look at using pandas to do some data analysis.